# AI Agent Crash Course Part 3
- Before you start, create a .env file and set the OPENAI_API_KEY as follows:

OPENAI_API_KEY="your-openai-api-key"

- Install Ollama: https://ollama.com/

In [ ]:
!pip install crewai ollama openai
!ollama pull llama3.2:1b

In [ ]:
import ollama
import openai
import os

from crewai.flow.flow import Flow, listen, start
from dotenv import load_dotenv
from IPython.display import Markdown

# Load environment variables (like API keys)
load_dotenv()

## Basis Flow with OpenAI

In [ ]:
openai_client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

class MovieRecommendationFlow(Flow):
    model = "gpt-4o-mini"

    @start()
    def generate_genre(self):

        response = openai_client.chat.completions.create(
            model=self.model,
            messages=[
                {
                    "role": "user",
                    "content": "Give me a random movie genre.",
                },
            ],
        )

        random_genre = response.choices[0].message.content.strip()
        return random_genre

    @listen(generate_genre)
    def recommend_movie(self, random_genre):
    
        response = openai_client.chat.completions.create(
            model=self.model,
            messages=[
                {
                    "role": "user",
                    "content": f"Recommend a highly rated movie in the {random_genre} genre.",
                },
            ],
        )

        movie_recommendation = response.choices[0].message.content.strip()
        return movie_recommendation

# Execute the flow
flow = MovieRecommendationFlow()
final_result = await flow.kickoff_async()

In [ ]:
flow.state

In [ ]:
from pprint import pprint
pprint(final_result)

## Basis Flow with Ollama

In [ ]:
class MovieRecommendationFlow(Flow):
    model = "llama3.2:1b"

    @start()
    def generate_genre(self):
        response = ollama.chat(
            model=self.model,
            messages=[
                {
                    "role": "user",
                    "content": "Give me a random movie genre.",
                },
            ],
        )

        random_genre = response.message.content
        return random_genre

    @listen(generate_genre)
    def recommend_movie(self, random_genre):
    
        response = ollama.chat(
            model=self.model,
            messages=[
                {
                    "role": "user",
                    "content": f"Recommend a highly rated movie in the {random_genre} genre.",
                },
            ],
        )

        movie_recommendation = response.message.content

        return movie_recommendation

# Execute the flow
flow = MovieRecommendationFlow()
final_result = await flow.kickoff_async()

In [ ]:
print(final_result)

## Unstructured States with Ollama—example 1

In [ ]:
class MovieRecommendationFlow(Flow):

    @start()
    def generate_genre(self):

        response = ollama.chat(
            model="llama3.2:1b",
            messages=[
                {
                    "role": "user",
                    "content": "Give me a random movie genre.",
                },
            ],
        )

        random_genre = response.message.content
        self.state["genre"] = random_genre

        return random_genre

    @listen(generate_genre)
    def recommend_movie(self, random_genre):
    
        response = ollama.chat(
            model="llama3.2:1b",
            messages=[
                {
                    "role": "user",
                    "content": f"Recommend a highly rated movie in the {random_genre} genre.",
                },
            ],
        )

        movie_recommendation = response.message.content
        self.state["recommendation"] = movie_recommendation
        return movie_recommendation

# Execute the flow
flow = MovieRecommendationFlow()
final_result = await flow.kickoff_async()

In [ ]:
Markdown(final_result)

In [ ]:
from pprint import pprint

pprint(flow.state)

## Structured States—example 2

In [ ]:
from crewai.flow.flow import Flow, listen, start

class TaskManagementFlow(Flow):

    @start()
    def generate_task(self):
        print(f"Flow started. State ID: {self.state['id']}\n")
        
        # Step 1: Generate a new task
        self.state["task"] = "Fix a critical bug in the payment system"
        self.state["status"] = "Pending"
        print(f"Task generated: {self.state['task']} (Status: {self.state['status']})\n")

    @listen(generate_task)
    def start_task(self):
        # Step 2: Update task status to 'In Progress'
        self.state["status"] = "In Progress"
        print(f"Task status updated: {self.state['status']}\n")

    @listen(start_task)
    def complete_task(self):
        # Step 3: Mark task as 'Completed'
        self.state["status"] = "Completed"
        print(f"Task status updated: {self.state['status']}\n")
        pprint(f"Final Task State: {self.state}")

# Execute the flow
flow = TaskManagementFlow()
final_result = await flow.kickoff_async()

## Structured States

In [13]:
from crewai.flow.flow import Flow, listen, start
from pydantic import BaseModel

# Defining a structured state model
class TaskState(BaseModel):
    task: str = "None"
    status: str = "None"

class StructuredTaskFlow(Flow[TaskState]):

    @start()
    def generate_task(self):
        print(f"Flow started. State ID: {self.state.id}\n")
        self.state.task = "Develop a new API endpoint"
        self.state.status = "Pending"
        print(f"Task generated: {self.state.task} (Status: {self.state.status})\n")

    @listen(generate_task)
    def start_task(self):
        self.state.status = "In Progress"
        print(f"Task status updated: {self.state.status}\n")

    @listen(start_task)
    def complete_task(self):
        self.state.status = "Completed"
        print(f"Task status updated: {self.state.status}\n")
        pprint(f"Final Task State: {self.state}")

In [ ]:
# Execute the flow
flow = StructuredTaskFlow()
final_result = await flow.kickoff_async()

## Adding a new field to the structured state model

In [15]:
from crewai.flow.flow import Flow, listen, start
from pydantic import BaseModel

# Defining a structured state model
class TaskState(BaseModel):
    task: str = "None"
    status: str = "None"

class StructuredTaskFlow(Flow[TaskState]):

    @start()
    def generate_task(self):
        print(f"Flow started. State ID: {self.state.id}\n")
        self.state.task = "Develop a new API endpoint"
        self.state.status = "Pending"
        self.state.priority = "High" # adding a new field that is not defined in the TaskState model
        print(f"Task generated: {self.state.task} (Status: {self.state.status})\n")

    @listen(generate_task)
    def start_task(self):
        print(f"Task status currently: {self.state.status}\n")
        self.state.status = "In Progress"
        print(f"Task status updated: {self.state.status}\n")

    @listen(start_task)
    def complete_task(self):
        self.state.status = "Completed"
        print(f"Task status updated: {self.state.status}\n")
        pprint(f"Final Task State: {self.state}")

In [ ]:
# Execute the flow and this should raise an error
flow = StructuredTaskFlow()
final_result = await flow.kickoff_async()

## Conditional Flows—OR

In [ ]:
from crewai.flow.flow import Flow, listen, or_, start

class SupportFlow(Flow):

    @start()
    def live_chat_request(self):
        return "Support request received via live chat"

    @start()
    def email_ticket_request(self):
        return "Support request received via email ticket"

    @listen(or_(live_chat_request, email_ticket_request))
    def log_request(self, request_source):
        print(f"Logging request: {request_source}")

# Execute the flow
flow = SupportFlow()
final_result = await flow.kickoff_async()

# Conditional Flows—AND

In [ ]:
from crewai.flow.flow import Flow, and_, listen, start

class TicketEscalationFlow(Flow):

    @start()
    def user_confirms_issue(self):
        self.state["user_confirmation"] = True
        print("User confirmed they still need assistance.")

    @listen(user_confirms_issue)
    def agent_reviews_ticket(self):
        self.state["agent_review"] = True
        print("Support agent has reviewed the ticket.")

    @listen(and_(user_confirms_issue, agent_reviews_ticket))
    def escalate_ticket(self):
        print("Escalating ticket to Level 2 support!")

# Execute the flow
flow = TicketEscalationFlow()
final_result = await flow.kickoff_async()

# Conditional Flows—Router

In [ ]:
import random
from crewai.flow.flow import Flow, listen, router, start
from pydantic import BaseModel

class TicketState(BaseModel):
    priority: str = "low"

class TicketRoutingFlow(Flow[TicketState]):

    @start()
    def classify_ticket(self):
        print("Classifying support ticket...")
        self.state.priority = random.choice(["high", "low"])
        print(f"Ticket classified as: {self.state.priority}")

    @router(classify_ticket)
    def route_ticket(self):
        return "urgent_support" if self.state.priority == "high" else "email_support"

    @listen("urgent_support")
    def assign_to_agent(self):
        print("Urgent ticket assigned to a live support agent!")

    @listen("email_support")
    def send_to_email_queue(self):
        print("Non-urgent ticket sent to email support queue.")

# Execute the flow
flow = TicketRoutingFlow()
final_result = await flow.kickoff_async()

# Crew with Flows

In [ ]:
!crewai create flow test_flow

- Add the API key to the .env file in the test_flow folder
- Also, edit the src/test_flow/crews/poem_crew/poem_crew.py file by adding this code:

import os

from dotenv import load_dotenv

load_dotenv()


In [ ]:
%cd test_flow/src

In [ ]:
!python test_flow/main.py